In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob
from IPython.display import clear_output
import face_recognition
import argparse
import dlib
import os
from utils.aux_functions import *
import math
import datetime
from RetinaFace.retinaface_cov import RetinaFaceCoV
from PIL import Image, ImageFile
import insightface
import tensorflow as tf
from parameters import *
import keras

from numpy import load
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
import pickle


%matplotlib inline

----
### MaskTheFace 마스크 생성

In [ ]:
### MaskTheFace Argument load & Generate Masked Image from Selected Folders ###

parser = argparse.ArgumentParser(
    description="MaskTheFace - Python code to mask faces dataset"
)

parser.add_argument(
    "--path",
    type=str,
    default="../users",
    help="Path to either the folder containing images or the image itself",
)

types = parser.add_argument(
    "--mask_type",
    type=str,
    default='cloth',
    choices=["surgical", "N95", "KN95", "cloth", "gas", "inpaint", "random", "all"],
    help="Type of the mask to be applied. Available options: all, surgical_blue, surgical_green, N95, cloth",
)

parser.add_argument(
    "--pattern",
    type=str,
    default="",
    help="Type of the pattern. Available options in masks/textures",
)

parser.add_argument(
    "--pattern_weight",
    type=float,
    default=0.5,
    help="Weight of the pattern. Must be between 0 and 1",
)

parser.add_argument(
    "--color",
    type=str,
    default="#0473e2",
    help="Hex color value that need to be overlayed to the mask",
)

parser.add_argument(
    "--color_weight",
    type=float,
    default=0.5,
    help="Weight of the color intensity. Must be between 0 and 1",
)

parser.add_argument(
    "--code",
    type=str,
    default="",
    help="Generate specific formats",
)

parser.add_argument(
    "--verbose", dest="verbose", action="store_true", help="Turn verbosity on"
)

parser.add_argument(
    "--write_original_image",
    dest="write_original_image",
    action="store_true",
    help="If true, original image is also stored in the masked folder",
)

masks = ["surgical", "N95", "KN95", "cloth"]

parser.set_defaults()

args = parser.parse_args(args=[])
args.write_path = args.path

# Set up dlib face detector and predictor
args.detector = dlib.get_frontal_face_detector()
path_to_dlib_model = "dlib_models/shape_predictor_68_face_landmarks.dat"
if not os.path.exists(path_to_dlib_model):
    download_dlib_model()

args.predictor = dlib.shape_predictor(path_to_dlib_model)

# Extract data from code
mask_code = "".join(args.code.split()).split(",")
args.code_count = np.zeros(len(mask_code))
args.mask_dict_of_dict = {}

is_directory, is_file, is_other = check_path(args.path)
display_MaskTheFace()

if is_directory:
    path, dirs, files = os.walk(args.path).__next__()
    file_count = len(files)
    dirs_count = len(dirs)
    if len(files) > 0:
        print_orderly("Masking image files", 60)

    # Process files in the directory if any
    for mask in masks:
        args.mask_type = mask
        for f in tqdm(files):
            image_path = path + "/" + f
            write_path = path
            if not os.path.isdir(write_path):
                os.makedirs(write_path)

            if is_image(image_path):
                # Proceed if file is image
                if args.verbose:
                    str_p = "Processing: " + image_path
                    tqdm.write(str_p)

                split_path = f.rsplit(".")
                masked_image, mask, mask_binary_array, original_image = mask_image(
                    image_path, args
                )
                for i in range(len(mask)):
                    w_path = (
                        write_path
                        + "/"
                        + split_path[0]
                        + "_"
                        + mask[i]
                        + "."
                        + split_path[1]
                    )
                    img = masked_image[i]
                    cv2.imwrite(w_path, img)

print("Processing Done")

----
### Face Embedding 추출

In [8]:
from numpy import dot
from numpy.linalg import norm

def triplet_loss(y_true, y_pred, alpha = ALPHA):
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, positive)), axis=-1)
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, negative)), axis=-1)
    basic_loss = tf.add(tf.subtract(pos_dist, neg_dist), alpha)
    loss = tf.reduce_sum(tf.maximum(basic_loss, 0.0))    
    return loss

def verify(image_path, identity, database, model):
    
    encoding = img_to_encoding2(image_path, model, False)
    min_dist = 1000
    for  pic in database:
        dist = np.linalg.norm(encoding - pic)
        if dist < min_dist:
            min_dist = dist
    print(identity + ' : ' +str(min_dist)+ ' ' + str(len(database)))
    
    if min_dist<THRESHOLD:
        door_open = True
    else:
        door_open = False
        
    return min_dist, door_open

def img_to_encoding2(image, model, path=True):
    img = image
    img = tf.keras.applications.inception_resnet_v2.preprocess_input(img)
    x_train = np.array([img])
    embedding = model.predict_on_batch(x_train)
    return embedding

def cos_sim(A, B):
    return dot(A,B) / (norm(A) * norm(B))


In [ ]:
#### 데이터베이스의 얼굴들이 충분히 존재하는 경우 svm을 이용한 classify를 진행

# in_encoder = Normalizer(norm='l2')
# out_encoder = LabelBinarizer()

# trainx = []
# trainy = []
# for name in faces:
#     ben_list = os.listdir('cropped/' + name)

#     for i in ben_list:
#         trainx.append(img_to_encoding2('cropped/' + name + '/' + i, FRmodel)[0])
#         trainy.append(name)
        
# testx = []
# testy = []
# for name in faces:
#     ben_list = os.listdir('val_set/' + name)

#     for i in ben_list:
#         testx.append(img_to_encoding2('val_set/' + name + '/' + i, FRmodel)[0])
#         testy.append(name)
        

# trainX = in_encoder.transform(trainx)
# testX = in_encoder.transform(testx)

# model = SVC(kernel='linear', probability=True)
# model.fit(trainx, trainy)

# yhat_test = model.predict_proba(testx)
# yhat_test

In [3]:
### DATABASE에 저장된 각 이미지들에서 face embedding을 추출하여 저장해놓음
### MaskTheFace를 통해 생성해놓은 이미지 또한 detection 및 feature extraction을 진행해야하므로 retinaface와 같은
### detection with mask 모델들의 도입이 필요함
### 추가적으로 face_recognition의 face embedding 결과가 동양인에 대해서는 다소 미흡한 부분이 있음으로 확인되어 다른 모델 탐구 필요
### RetinaFace Covid Load and Find Face Bounding Box
thresh = 0.8
mask_thresh = 0.2
count = 1
gpuid = 0
detector = RetinaFaceCoV('./models/cov2/mnet_cov2', 0, -1, 'net3l')

print("Trained model found")
print("Loading custom trained model...")
model = keras.models.load_model('models/facenet_keras.h5', custom_objects={'triplet_loss': triplet_loss})        
print('Completed')
# model = insightface.model_zoo.get_model('arcface_r100_v1')
# model.prepare(ctx_id = -1)

Trained model found
Loading custom trained model...
Completed


In [16]:
known_face_encodings = []
known_face_names = []

files = glob.glob('../users/*.jpg')

for file in files:
    scales = [640, 1080]
    img = cv2.imread(file)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    img = cv2.resize(img, (640, 480), interpolation=cv2.INTER_CUBIC)
    
    im_shape = img.shape
    target_size = scales[0]
    max_size = scales[1]
    im_size_min = np.min(im_shape[0:2])
    im_size_max = np.max(im_shape[0:2])
    #im_scale = 1.0
    #if im_size_min>target_size or im_size_max>max_size:
    im_scale = float(target_size) / float(im_size_min)
    # prevent bigger axis from being more than max_size:
    if np.round(im_scale * im_size_max) > max_size:
        im_scale = float(max_size) / float(im_size_max)

    scales = [im_scale]
    flip = False
    
    faces, landmarks = detector.detect(img, thresh, scales=scales, do_flip=flip)
    
    if len(faces) != 0:
        box = faces[0][0:4].astype(np.int)
        cv2.rectangle(img, (box[0], box[1]), (box[2], box[3]), (0,0,255), 2)
        croped_img = img[box[1]:box[3], box[0]:box[2]]
        croped_img = cv2.resize(croped_img, (160, 160), cv2.INTER_CUBIC)
        emb = img_to_encoding2(croped_img, model)[0]
        emb = emb.reshape(-1)
        known_face_encodings.append(emb)
        known_face_names.append(file.split('/')[2][:-4])
        print(file[8:-4])

# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True


/hyungwoo
/sanggyu
/sangrae
/biden
/obama


In [31]:
video = cv2.VideoCapture('http://posproject201013.iptime.org:8787/video_feed')

fourcc = cv2.VideoWriter_fourcc(*'DIVX')
out = cv2.VideoWriter('output2.avi', fourcc, 25.0, (640,480))

while True:
    #_, img = video.read()
    scales = [640, 1080]
    _, img = video.read()

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    im_shape = img.shape
    
    target_size = scales[0]
    max_size = scales[1]
    im_size_min = np.min(im_shape[0:2])
    im_size_max = np.max(im_shape[0:2])
    #im_scale = 1.0
    #if im_size_min>target_size or im_size_max>max_size:
    im_scale = float(target_size) / float(im_size_min)
    # prevent bigger axis from being more than max_size:
    if np.round(im_scale * im_size_max) > max_size:
        im_scale = float(max_size) / float(im_size_max)

    scales = [im_scale]
    flip = False

    faces, landmarks = detector.detect(img, thresh, scales=scales, do_flip=flip)
    if len(faces) != 0:
        for i in range(faces.shape[0]):
            face = faces[i]
            box = face[0:4].astype(np.int)
            mask = face[5]
            
            name = "Unknown"
            
            if mask>=mask_thresh:
                name = 'Mask Detected. Please take off your mask.'
                ###      R G B
                color = (0,0,255)
            else:
                color = (0,255,0)
                croped_img = img[box[1]:box[3], box[0]:box[2]]
                croped_img = cv2.resize(croped_img, (160, 160), cv2.INTER_CUBIC)
                emb = img_to_encoding2(croped_img, model)[0]
                emb = emb.reshape(-1)
                sim = -1
                idx = 0
                for i, known_face in enumerate(known_face_encodings):
                    cur_sim = cos_sim(known_face, emb)
                    if sim < cur_sim:
                        sim = cur_sim
                        idx = i
                
                if sim > 0.5:
                    name = known_face_names[idx]

            cv2.rectangle(img, (box[0], box[1]), (box[2], box[3]), color, 2)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(img, name, (box[0] + 6, box[1] - 6), font, 1.0, (255, 255, 255), 1)

#             landmark5 = landmarks[i].astype(np.int)
            
#             for l in range(landmark5.shape[0]):
#                 color = (255,0,0)
#                 cv2.circle(img, (landmark5[l][0], landmark5[l][1]), 1, color, 2)
    plt.axis('off')
    plt.imshow(img)
    plt.show()
    cv2.waitKey(1)
    clear_output(wait=True)
    
    out.write(cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
            
cap.release()
out.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

----
### 웹서버 비디오 스트림 재생 및 Mask Detection

In [ ]:
video = cv2.VideoCapture('http://posproject201013.iptime.org:8787/video_feed')

while True:
    _, frame = video.read()

    origin = frame
        # Resize frame of video to 1/4 size for faster face recognition processing
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_small_frame = small_frame[:, :, ::-1]

    # Find all the faces and face encodings in the current frame of video
    face_locations = face_recognition.face_locations(rgb_small_frame, model='cnn')
    face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

    face_names = []
    for face_encoding in face_encodings:
        # See if the face is a match for the known face(s)
        # [ [featur1], [featur2]] 2 * 512 * 1
        # [feature_enco] 512 * 1
        # [ True, False]
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        
        name = "Unknown"

        # Or instead, use the known face with the smallest distance to the new face
        face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
        best_match_index = np.argmin(face_distances)
        if matches[best_match_index]:
            name = known_face_names[best_match_index]

        face_names.append(name)

    # Display the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
        
    masked_image, mask, mask_binary_array, original_image = mask_image_file(origin, args)
    
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    plt.axis('off')
    
   
    if len(masked_image):
        frame = cv2.cvtColor(masked_image[0], cv2.COLOR_BGR2RGB)
        plt.imshow(frame)
    else:
        plt.imshow(frame)
    
    plt.show()
    cv2.waitKey(1)
    clear_output(wait=True)

In [ ]:
def url_to_image(url):
    resp = urllib.request.urlopen(url)
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    # return the image
    return image

url = 'https://github.com/deepinsight/insightface/raw/master/deploy/Tom_Hanks_54745.png'
img = url_to_image(url)

print(img.shape)